# Extender

In [ ]:
using BSON: @save, @load
using Flux
using Plots

include("src/Extender.jl")

using .Extender

## Parameters initialization

To extend a neural network, you first need to initialize parameters for the input model, the desired output model, and general parameters. These general parameters include `epsilon`, a small value used instead of zero for initializing new weights and biases. This is crucial to avoid getting stuck in gradient descent, as zero initialization leads to zero gradients, making weight updates impossible.

**Input/Output Model Parameters:**

*   **`file_name`:** The name of the file used for loading the input model or saving the output model.
*   **`structure`:** A vector representing the number of neurons in each layer. 
*   **`activations`:** A vector containing the names of activation functions for each layer (e.g., "identity", "relu").
*   **`use_bias`:** A boolean value indicating whether the neural network uses biases.
*   **`precision`:** The precision of the neural network's weights (e.g., "f64", "f32", or "f16").

**General Parameters:**

*   **`set_weights_to`:**  A small value (`epsilon`) used for initializing the weights of newly added neurons. 
*   **`test_samples`:** The number of samples used to test the extended model with randomly generated input.

In [ ]:
# Input Model
input_nn_params = NeuralNetParams(
    "example-methanol-model.bson",
    [8, 30, 30, 30, 1],
    ["identity", "relu", "relu", "identity"],
    false,
    "f64"
)

# Output Model
output_nn_params = NeuralNetParams(
    "output-example-model.bson",
    [10, 35, 35, 35, 10, 1],
    ["identity", "relu", "relu", "identity", "identity"],
    true,
    "f32"
)

# General Parameters
general_params = GeneralParams(0.001, 10000);

In [ ]:
input_model = Extender.load_model(input_nn_params.file_name);

## Run Extending of Input model

In [ ]:
output_model = Extender.run_extending(input_model, input_nn_params, output_nn_params, general_params);

### Saving of Extended model

In [ ]:
Extender.save_model(output_nn_params.file_name, output_model)

## Weights heatmaps

In [ ]:
input_model

In [ ]:
Extender.plot_model_parameters(input_model)

In [ ]:
output_model

In [ ]:
Extender.plot_model_parameters(output_model)

# Manual Testing

In [ ]:
scale = 1
n1 = size(input_model.layers[1].weight)[2]
n2 = size(output_model.layers[1].weight)[2]

input_vector1 = rand(n1) .* scale
v_2 = rand(n2 - n1) .* scale
input_vector2 = vcat(input_vector1, v_2)
result1 = input_model(input_vector1)
result2 = output_model(input_vector2)
difference = abs(result1[1] - result2[1]) / abs(result1[1]) * 100
println("Input model result: $(result1[1])")
println("Output model result: $(result2[1])")
println("Relative difference: $(difference) %")